<a href="https://colab.research.google.com/github/41371125h-chinrouzhen/114-1-PL/blob/main/HW2_%E6%88%90%E7%B8%BE%E4%B8%80%E6%9C%AC%E9%80%9A%EF%BC%88AI%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **成績一本通**
目標：從 Sheet 讀各次作業/測驗 → 計平時/期末/總成績 → 回寫成績總表。

AI 點子：針對「開放式作答」欄位，用模型產出整體摘要與常見迷思整理（不評分，只做總結）。

Google sheet鏈接:https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing

注記：API key不可用，無法生成相關總結


In [ ]:
# 套件與認證
import pandas as pd
import numpy as np
import gspread
from google.colab import auth
from google.auth import default
import time
from datetime import datetime
import getpass

auth.authenticate_user()
creds, _ = default()
client = gspread.authorize(creds)

# Google Sheet
SHEET_URL = "https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing"
RECORDS_SHEET = "Records"
SUMMARY_SHEET = "Summary"

# Reecord
def read_records_df():
    ws = client.open_by_url(SHEET_URL).worksheet(RECORDS_SHEET)
    data = ws.get_all_values()
    if not data:
        return pd.DataFrame()
    df = pd.DataFrame(data[1:], columns=data[0])
    numeric_cols = ["作業1","作業2","作業3","小測1","小測2","期末"]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
    if '開放式作答' in df.columns:
        df['開放式作答'] = df['開放式作答'].fillna("").astype(str)
    else:
        df['開放式作答'] = ""
    return df

df = read_records_df()

# Cal
homework_cols = ["作業1","作業2","作業3"]
quiz_cols = ["小測1","小測2"]
df['平時成績'] = df[homework_cols + quiz_cols].mean(axis=1)
df['總成績'] = df['平時成績']*0.4 + df['期末']*0.6


# 寫回
try:
    summary_ws = client.open_by_url(SHEET_URL).worksheet(SUMMARY_SHEET)
    summary_ws.clear()
except gspread.WorksheetNotFound:
    summary_ws = client.open_by_url(SHEET_URL).add_worksheet(title=SUMMARY_SHEET, rows="100", cols="20")

summary_ws.update([df.columns.values.tolist()] + df.values.tolist())
print(f"已更新")



已更新


In [12]:
!pip install --upgrade google-generativeai
!pip install google-generativeai
!pip install gradio
!pip install gradio

In [29]:
import pandas as pd
from google.colab import auth, userdata
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import sys

In [14]:
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    sheets_enabled = True
    print("Google Sheets 認證成功。")
except Exception as e:
    print(f"Google Sheets 認證失敗: {e}")
    sheets_enabled = False
    gc = None

Google Sheets 認證成功。


In [34]:
# Gemini API
api_key = ''
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.5-flash')
print("Gemini API 已成功設定。")


Gemini API 已成功設定。


In [35]:
response = model.generate_content('1+1')
print(response.text)


DefaultCredentialsError: 
  No API_KEY or ADC found. Please either:
    - Set the `GOOGLE_API_KEY` environment variable.
    - Manually pass the key with `genai.configure(api_key=my_api_key)`.
    - Or set up Application Default Credentials, see https://ai.google.dev/gemini-api/docs/oauth for more information.

In [36]:
SHEET_URL = "https://docs.google.com/spreadsheets/d/1HA1cvniLmO1O362GtHlrLEK3TQ8_XQJBULMpbaU-29s/edit?usp=sharing"
worksheet_name="1"
spreadsheet = gc.open_by_url(SHEET_URL)
worksheet = spreadsheet.worksheet(worksheet_name)
data = worksheet.get_all_records()
df = pd.DataFrame(data)

data_summary = f"""
請分析以下學生成績資料：

資料概況：
- 總筆數：{len(df)}
- 欄位：{', '.join(df.columns.tolist())}

資料內容：
{df.to_string()}

請提供以下分析：
1. 整體成績表現摘要
2. 各科目的平均成績和分布情況
3. 學生的強弱科目分析
4. 學習建議和改進方向
5. 需要特別關注的科目或學期

請以清楚、結構化的方式呈現分析結果。
"""


response = model.generate_content(data_summary)
print(response.text)

WorksheetNotFound: 1